# Azure ML jobs demo: PyTorch - Wine Quality Prediction Model Training

In this notebook learn how to use Azure Machine Learning job submission interface to train a PyTorch neural network model. 

In this example, we will use the wine quality dataset as demonstrated in the PyTorch demo (if you haven't completed it, no worries). We will repeat the pipeline created in the previous, except in this version we'll submit the job to complete in the background. 

The goal is to predict wine quality (score 0-10) based on physicochemical properties like acidity, sugar content, alcohol level, etc.

## Learning Objectives:
- Setup the Azure ML client connection
- Define and save a conda .yaml file
- Create a training script
- Configure and submit a job
- Monitor and view the jobs output

## Prerequisites
You need a compute instance to run the code as it relies upon a custom environment that is not available with "Serverless Spark Compute". If you don't have a compute instance, select **Create compute** on the toolbar to first create one.  You can use all the default settings. 

## Set your kernel

* If your compute instance is stopped, start it now.  
        
* Once your compute instance is running, make sure the that the kernel, found on the top right, is `Python 3.10 - AzureML`.  If not, use the dropdown to select this kernel.

## Background -- Command Jobs in Azure Machine Learning

To train a model, you need to submit a *job*. The type of job you'll submit in this tutorial is a *command job*. Azure Machine Learning offers several different types of jobs to train models. Users can select their method of training based on complexity of the model, data size, and training speed requirements.  In this tutorial, you'll learn how to submit a *command job* to run a *training script*. 

A command job is a function that allows you to submit a custom training script to train your model. This can also be defined as a custom training job. A command job in Azure Machine Learning is a type of job that runs a script or command in a specified environment. You can use command jobs to train models, process data, or any other custom code you want to execute in the cloud. 

In this tutorial, we'll focus on using a command job to create a custom training job that we'll use to train a model. For any custom training job, the below items are required:

* compute resource (usually a compute cluster)
* environment
* data
* command job 
* training script


## 1. Create handle to workspace

Before we dive in the code, you need a way to reference your workspace. You'll create `ml_client` for a handle to the workspace.  You'll then use `ml_client` to manage resources and jobs.

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find these values:

1. In the upper right Azure Machine Learning studio toolbar, select your workspace name.
2. Copy the value for workspace, resource group and subscription ID into the code.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

subscription_id = "SUBSCRIPTION ID HERE"
resource_group = "RESOURCE NAME HERE"
workspace_name = "WORKSPACE NAME HERE"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

#### NOTE!

Creating MLClient will not connect to the workspace. The client initialisation is lazy, it will wait for the first time it needs to make a call (this will happen in the next code cell).

In [ ]:
# Verify that the handle works correctly.
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location, ":", ws.resource_group)

## Create a job environment

To run your Azure Machine Learning job on your compute resource, you need an environment. An environment lists the software runtime and libraries that you want installed on the compute where you’ll be training. It's similar to your python environment on your local machine or the kernel running in this notebook currently.

Azure Machine Learning provides many curated or ready-made environments, which are useful for common training and inference scenarios. 

In this example, you'll create a custom conda environment for your jobs, using a conda yaml file.

First, create a directory to store the file in.

In [ ]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

The cell below uses IPython magic to write the conda file into the directory you just created.

In [ ]:
%%writefile {dependencies_dir}/conda.yaml
name: pytorch-env
channels:
  - conda-forge
  - pytorch
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=1.0.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pytorch>=1.11.0
  - torchvision
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - mlflow==2.8.0
    - mlflow-skinny==2.8.0
    - azureml-mlflow==1.51.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib
    - seaborn
    - azureml-fsspec


The specification contains some usual packages, that you'll use in your job (numpy, pip).

Reference this *yaml* file to create and register this custom environment in your workspace:

Below we package all the componments of the envrionemtn together using the [Environment Class](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.environment?view=azure-python). We give it a name, a description, tags, as well as the path to the .yaml file and an operating system image to build upon. In this case we're using a Ubuntu Linux image. 

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-pytorch"

custom_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for PyTorch Wine Quality job",
    tags={"pytorch": "1.11.0"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"Environment with name {custom_job_env.name} is registered to workspace, the environment version is {custom_job_env.version}"
)

## Configure a training job using the command function

You create an Azure Machine Learning *command job* to train a model for credit default prediction. The command job runs a *training script* in a specified environment on a specified compute resource.  You've already created the environment and the compute cluster.  Next you'll create the training script. In our specific case, we're training our dataset to produce a classifier using the `GradientBoostingClassifier` model. 

The *training script* handles the data preparation, training and registering of the trained model. The method `train_test_split` handles splitting the dataset into test and training data. In this tutorial, you'll create a Python training script. 

Command jobs can be run from CLI, Python SDK, or studio interface. In this tutorial, you'll use the Azure Machine Learning Python SDK v2 to create and run the command job.

## Create training script

Let's start by creating the training script - the *main.py* python file.

First create a source folder for the script:

In [ ]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

This script handles the preprocessing of the data, splitting it into test and train data. It then consumes this data to train a tree based model and return the output model. 

[MLFlow](https://learn.microsoft.com/articles/machine-learning/concept-mlflow) is used to log the parameters and metrics during our job. The MLFlow package facilitates the running and logging of metrics and results for each model Azure trains.

In [ ]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch

class WineQualityNet(nn.Module):
    def __init__(self, input_dim):
        super(WineQualityNet, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
        
        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.2)
    parser.add_argument("--learning_rate", required=False, default=0.001, type=float)
    parser.add_argument("--batch_size", required=False, default=32, type=int)
    parser.add_argument("--epochs", required=False, default=100, type=int)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.pytorch.autolog()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    print(' ')

    # Load wine quality dataset
    wine_data = pd.read_csv(args.data)

    print(wine_data)
    print(' ')
    print('Columns:')
    print(wine_data.columns.tolist())
    print(' ')
    
    mlflow.log_metric("num_samples", wine_data.shape[0])
    mlflow.log_metric("num_features", wine_data.shape[1] - 1)

    # Separate features and target
    X = wine_data.drop('quality', axis=1).values
    y = wine_data['quality'].values

    # Split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=args.test_train_ratio, random_state=42)
    
    # Further split training data into train and validation
    X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_final)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train_final)
    X_val_tensor = torch.FloatTensor(X_val_scaled)
    y_val_tensor = torch.FloatTensor(y_val)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    y_test_tensor = torch.FloatTensor(y_test)

    # Create PyTorch datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size)

    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    print(f"Number of features: {X_train_final.shape[1]}")
    ####################
    #</prepare the data>
    ####################

    ##################
    #<train the model>
    ##################
    # Initialize model
    model = WineQualityNet(X_train_final.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # Lists to store losses for plotting
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(args.epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X).squeeze()
                loss = criterion(outputs, batch_y)
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{args.epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_history.png')
    plt.show()

    # Evaluation on test set
    model.eval()
    with torch.no_grad():
        test_predictions = []
        test_targets = []
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X).squeeze()
            test_predictions.extend(outputs.numpy())
            test_targets.extend(batch_y.numpy())

    # Convert to numpy arrays
    predictions = np.array(test_predictions)
    y_test_array = np.array(test_targets)

    # Calculate metrics
    mse = mean_squared_error(y_test_array, predictions)
    r2 = r2_score(y_test_array, predictions)
    rmse = np.sqrt(mse)

    print(f"Test Set Performance:")
    print(f'Test MSE: {mse:.4f}')
    print(f'Test RMSE: {rmse:.4f}')
    print(f'Test R² Score: {r2:.4f}')

    # Visualization of predictions
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_array, predictions, alpha=0.5)
    plt.plot([y_test_array.min(), y_test_array.max()], [y_test_array.min(), y_test_array.max()], 'r--', lw=2)
    plt.xlabel('Actual Quality')
    plt.ylabel('Predicted Quality')
    plt.title('Actual vs Predicted Wine Quality')
    plt.grid(True)
    plt.savefig('predictions_scatter.png')
    plt.show()

    # Distribution of prediction errors
    errors = predictions - y_test_array
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=30, edgecolor='black')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    plt.title('Distribution of Prediction Errors')
    plt.grid(True, alpha=0.3)
    plt.savefig('error_distribution.png')
    plt.show()

    # Log metrics
    mlflow.log_metric("test_mse", mse)
    mlflow.log_metric("test_rmse", rmse)
    mlflow.log_metric("test_r2", r2)
    mlflow.log_param("learning_rate", args.learning_rate)
    mlflow.log_param("batch_size", args.batch_size)
    mlflow.log_param("epochs", args.epochs)
    
    # Log the plots as artifacts
    mlflow.log_artifact('training_history.png')
    mlflow.log_artifact('predictions_scatter.png')
    mlflow.log_artifact('error_distribution.png')
    ###################
    #</train the model>
    ###################

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.pytorch.log_model(
        pytorch_model=model,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.pytorch.save_model(
        pytorch_model=model,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

In this script, once the model is trained, the model file is saved and registered to the workspace. Registering your model allows you to store and version your models in the Azure cloud, in your workspace. Once you register a model, you can find all other registered model in one place in the Azure Studio called the model registry. The model registry helps you organize and keep track of your trained models. 

## Configure the command

Now that you have a script that can perform the classification task, use the general purpose **command** that can run command line actions. This command line action can be directly calling system commands or by running a script. 

Here, create input variables to specify the input data, split ratio, learning rate and registered model name.  The command script will:
* Use the environment created earlier - you can use the `@latest` notation to indicate the latest version of the environment when the command is run.
* Configure the command line action itself - `python main.py` in this case. The inputs/outputs are accessible in the command via the `${{ ... }}` notation.
* Since a compute resource was not specified, the script will be run on a [serverless compute cluster](https://learn.microsoft.com/azure/machine-learning/how-to-use-serverless-compute?view=azureml-api-2&tabs=python) that is automatically created.

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

registered_model_name = "wine_quality_model"
data_asset = ml_client.data.get("wine_quality_kaggle", version="1")

job = command(
    inputs=dict(
        data=Input(
            type=AssetTypes.URI_FILE,
            path=data_asset.path,
        ),
        test_train_ratio=0.2,
        learning_rate=0.001,
        batch_size=32,
        epochs=100,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --batch_size ${{inputs.batch_size}} --epochs ${{inputs.epochs}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="aml-pytorch@latest",
    display_name="wine_quality_prediction",
)

## Submit the job 

It's now time to submit the job to run in Azure Machine Learning studio. This time you'll use `create_or_update`  on `ml_client`.

In [ ]:
ml_client.create_or_update(job)

## View job output and wait for job completion

When you run the cell, the notebook output shows a link to the job's details page on Azure Studio. Alternatively, you can also select Jobs on the left navigation menu. A job is a grouping of many runs from a specified script or piece of code. Information for the run is stored under that job. The details page gives an overview of the job, the time it took to run, when it was created, etc. The page also has tabs to other information about the job such as metrics, Outputs + logs, and code. Listed below are the tabs available in the job's details page:


#### IMPORTANT
- There will be two jobs, first `prepare-image` which will be the job that spin's up your virtual Ubuntu machine. The second will have your experiment name, in this example `ml-demo`. The second can only run after the first is completed.
- The job will take 2 to 3 minutes to run. It could take longer (up to 10 minutes) if the compute cluster has been scaled down to zero nodes and custom environment is still building. Once completed you can explore the outputs


#### Jobs Output - main tabs

* Overview: The overview section provides basic information about the job, including its status, start and end times, and the type of job that was run
* Metrics: The metrics tab showcases key performance metrics from your model such as training score, f1 score, and precision score. 
* Images: This tab contains any images, such as confusion matrices, ROC curves, ect, that you have created during your run.
* Outputs + logs: The Outputs + logs tab contains logs generated while the job was running. This tab assists in troubleshooting if anything goes wrong with your training script or model creation.


### Stop compute instance

If you're not going to use it now, stop the compute instance:

1. In the studio, in the left navigation area, select **Compute**.
1. In the top tabs, select **Compute instances**
1. Select the compute instance in the list.
1. On the top toolbar, select **Stop**.
